# Opening a Distillery in the Finger Lakes Region of New York 

 ## Analysis for IBM Data Science Cert Capstone Project 

## Introduction: Business Problem

An alcoholic beverage distiller wants to open a new distillery in the Finger Lakes region of New York. The Finger Lakes region of Western New York is a very popular destination for tourism - especially for visiting vineyards, breweries and natural geographic features, such as gorges and waterfalls. The Finger Lakes region is very large and mostly rural. The distiller wants to find a location that will be close to similar businesses like wineries, and breweries, but also be located in an area that already gets a good amount of tourism activity.

The data science methodology tools will be used to identify and rank potential locations within the Finger Lakes region that are close to the most popular attractions while also being close to vineyards, wineries and breweries. The stakeholders prefer a location that does not already have existing distilleries.

## Data

In order to identify the most desirable areas, based upon the distiller's preferences, we will need to analyze: 
 - the location of vineyards, wineries, breweries, and tourist attractions
 - the density of preferred venue types within each area
 - the popularity ranking of venues within each area, to be derived from Foursquare user activity

The Foursquare API will be used to provide venue information, location data, and venue popularity data. To support this analysis we will need the following data from the Foursquare API:

    - name
    - categories
    - latitude, longitude
    - number of check-ins
    - number of tips
    - trending data 

Data available from the Foursquare API is incomplete related to licensed wineries, breweries and distilleries in the New York, Finger Lakes Region. Therefore, we will also use location data extracted from the https://data.ny.gov webpage which provides information regarding the name, address, type, and location of all New York licenses for alcohol production. The venues that are not listed by Foursquare will not have data to support popularity analysis.

Location data from the Mapquest Geo-coding API will be used to find the latitude and longitude of towns to be used in clustering analysis.



## Initialize our environment and build the datasets

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
pd.options.display.max_rows = 350
import numpy as np # library to handle data in a vectorized manner
import folium # plotting library
#import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
#import seaborn as sns
#%matplotlib inline 
import json
import csv
import os
from sklearn.cluster import KMeans 

# libraries for displaying images
#from IPython.display import Image 
#from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# libraries for html connection
import lxml
import html5lib
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Foursquare API constant variables
CLIENT_ID = 'XXXXXXXXXX'
CLIENT_SECRET = 'xxxxxxxxxx'
VERSION = '20190828'
# Mapquest API Key
mq_key = 'xXxXxXxXx'


## Setup the area of the Finger Lakes we are interested in

In [3]:
# The lat/Long for Ovid NY
latitude = 42.6763779
longitude = -76.8229294
# We're interesting in a radious of about 26 miles from Ovid NY
radius = 81000
LIMIT = 500

## Get data from the Foursquare API

In [4]:
fsq_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)   

In [5]:
results = requests.get(fsq_url).json()

## Let's see how many venues we have available to us from Foursquare

In [6]:
'There are {} venues within 50 miles of Ovid NY.'.format(len(results['response']['groups'][0]['items']))

'There are 100 venues within 50 miles of Ovid NY.'

In [7]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5310c5fc11d2285dadc69f6e',
  'name': 'Boundary Breaks Vineyard',
  'location': {'address': '1568 Porter Corver Rd',
   'lat': 42.628843745200676,
   'lng': -76.86429977416991,
   'labeledLatLngs': [{'label': 'display',
     'lat': 42.628843745200676,
     'lng': -76.86429977416991}],
   'distance': 6282,
   'postalCode': '14860',
   'cc': 'US',
   'city': 'Lodi',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['1568 Porter Corver Rd',
    'Lodi, NY 14860',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d14b941735',
    'name': 'Winery',
    'pluralName': 'Wineries',
    'shortName': 'Winery',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/winery_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-5310c5fc11d2

### Limitations of the Foursquare API

The data available from the Foursquare API  around Finger Lakes venues is too limited for our needs. So, we'll build a dataset that shows traffic around popular venues within the Foursquare database. Foursquare eliminated checkin data from the API so we'll use a combination of the number of photos and tips for each venues to show how popular each venue is. Photo and tip counts are premium calls to the API and are only available when getting the details of a single venue. We'll first build a dataset containing the venue information and venue id; then we'll make calls to get the photo and tip counts from each of the available venues.

### Build the summary Foursquare venue dataframe

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
# flatten the JSON
dataframe = json_normalize(items) 

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Boundary Breaks Vineyard,Winery,1568 Porter Corver Rd,42.628844,-76.864300,"[{'label': 'display', 'lat': 42.62884374520067...",6282,14860,US,Lodi,NY,United States,"[1568 Porter Corver Rd, Lodi, NY 14860, United...",NaN,5310c5fc11d2285dadc69f6e
1,Herman J Wiemer Vineyard,Vineyard,3962 NYS Rte 14,42.572324,-76.928225,"[{'label': 'display', 'lat': 42.57232383635794...",14441,14837,US,Dundee,NY,United States,"[3962 NYS Rte 14 (at Chestnut Rd), Dundee, NY ...",at Chestnut Rd,4b896854f964a520513332e3
2,Goose Watch Winery,Winery,5480 State Route 89,42.758744,-76.771589,"[{'label': 'display', 'lat': 42.75874440701328...",10084,14541,US,Romulus,NY,United States,"[5480 State Route 89, Romulus, NY 14541, Unite...",NaN,4bcf4e6841b9ef3bd72bf8e5
3,Cayuga Lake Creamery,Ice Cream Shop,8421 State Route 89,42.616514,-76.695925,"[{'label': 'display', 'lat': 42.61651416870882...",12351,14847,US,Interlaken,NY,United States,"[8421 State Route 89, Interlaken, NY 14847, Un...",NaN,4bbcdf8aa0a0c9b6e9ea1a0f
4,Americana Vineyards & Winery,Vineyard,4367 E Covert Rd,42.575900,-76.677117,"[{'label': 'display', 'lat': 42.57590024571664...",16362,14847,US,Interlaken,NY,United States,"[4367 E Covert Rd, Interlaken, NY 14847, Unite...",NaN,4bef155bea570f4750e38ed2
5,Aurora Inn,Bed & Breakfast,391 Main St,42.754605,-76.702821,"[{'label': 'display', 'lat': 42.75460493021212...",13127,13026,US,Aurora,NY,United States,"[391 Main St, Aurora, NY 13026, United States]",NaN,4bc41c07dce4eee1f3f1719d
6,Lamoreaux Landing Wine Cellars,Vineyard,9224 State Route 414,42.576636,-76.858460,"[{'label': 'display', 'lat': 42.57663558767305...",11478,14860,US,Lodi,NY,United States,"[9224 State Route 414, Lodi, NY 14860, United ...",NaN,4c5dcaa785a1e21e12825a11
7,Seneca Farms,Ice Cream Shop,2484 State Rt 54A,42.655121,-77.071898,"[{'label': 'display', 'lat': 42.65512050179350...",20516,14527,US,Penn Yan,NY,United States,"[2484 State Rt 54A, Penn Yan, NY 14527, United...",NaN,4bafdffff964a5200e283ce3
8,Scale House Brewery,Brewery,5930 NY-414,42.520547,-76.872937,"[{'label': 'display', 'lat': 42.52054731619821...",17824,14841,US,Hector,NY,United States,"[5930 NY-414, Hector, NY 14841, United States]",NaN,5920940aee628b240b5e41e1
9,Taughannock Falls State Park,State / Provincial Park,2221 Taughannock Road,42.542489,-76.606600,"[{'label': 'display', 'lat': 42.542489, 'lng':...",23157,14886,US,Trumansburg,NY,United States,"[2221 Taughannock Road, Trumansburg, NY 14886,...",NaN,4b57a728f964a520053a28e3


In [13]:
# these are the columns we'll use
venue_df = dataframe_filtered[['name', 'categories', 'city', 'lat', 'lng', 'id']]
venue_df.insert(5, "traffic","0")
venue_df.set_index('id')

,name,categories,city,lat,lng,traffic
id,,,,,,
5310c5fc11d2285dadc69f6e,Boundary Breaks Vineyard,Winery,Lodi,42.628844,-76.864300,0
4b896854f964a520513332e3,Herman J Wiemer Vineyard,Vineyard,Dundee,42.572324,-76.928225,0
4bcf4e6841b9ef3bd72bf8e5,Goose Watch Winery,Winery,Romulus,42.758744,-76.771589,0
4bbcdf8aa0a0c9b6e9ea1a0f,Cayuga Lake Creamery,Ice Cream Shop,Interlaken,42.616514,-76.695925,0
4bef155bea570f4750e38ed2,Americana Vineyards & Winery,Vineyard,Interlaken,42.575900,-76.677117,0
4bc41c07dce4eee1f3f1719d,Aurora Inn,Bed & Breakfast,Aurora,42.754605,-76.702821,0
4c5dcaa785a1e21e12825a11,Lamoreaux Landing Wine Cellars,Vineyard,Lodi,42.576636,-76.858460,0
4bafdffff964a5200e283ce3,Seneca Farms,Ice Cream Shop,Penn Yan,42.655121,-77.071898,0
5920940aee628b240b5e41e1,Scale House Brewery,Brewery,Hector,42.520547,-76.872937,0


### Let's have a look at what Foursquare gave us in the initial venue dataframe

In [14]:
venue_df

,name,categories,city,lat,lng,traffic,id
0,Boundary Breaks Vineyard,Winery,Lodi,42.628844,-76.864300,0,5310c5fc11d2285dadc69f6e
1,Herman J Wiemer Vineyard,Vineyard,Dundee,42.572324,-76.928225,0,4b896854f964a520513332e3
2,Goose Watch Winery,Winery,Romulus,42.758744,-76.771589,0,4bcf4e6841b9ef3bd72bf8e5
3,Cayuga Lake Creamery,Ice Cream Shop,Interlaken,42.616514,-76.695925,0,4bbcdf8aa0a0c9b6e9ea1a0f
4,Americana Vineyards & Winery,Vineyard,Interlaken,42.575900,-76.677117,0,4bef155bea570f4750e38ed2
5,Aurora Inn,Bed & Breakfast,Aurora,42.754605,-76.702821,0,4bc41c07dce4eee1f3f1719d
6,Lamoreaux Landing Wine Cellars,Vineyard,Lodi,42.576636,-76.858460,0,4c5dcaa785a1e21e12825a11
7,Seneca Farms,Ice Cream Shop,Penn Yan,42.655121,-77.071898,0,4bafdffff964a5200e283ce3
8,Scale House Brewery,Brewery,Hector,42.520547,-76.872937,0,5920940aee628b240b5e41e1
9,Taughannock Falls State Park,State / Provincial Park,Trumansburg,42.542489,-76.606600,0,4b57a728f964a520053a28e3


Upon reviewing the rows returned from Foursquare, we can see that two venues have NaN values for the 'city' column. One of these is a duplicate as it is just a differnet name for an existing venue - Watkins Glen State Park. The other is a restaurant which falls under our 'other' category. After searching for this retaurant on Google, we see that it is actually a restaurant and vinyard - this means it has some greater value to our analysis. So we'll manually remove the duplicate row for Watkins Glen State Park and manually update the 'city' and 'categories' values for Bully Hill Restaurant.

In [15]:
# this isn't the best way to accomplish this but it's the quickest
venue_df.drop([30], axis=0, inplace=True)

In [16]:
# just to confirm that the row for 'Watkins Glen State Park - Upper Entrance' is gone 
# and reconfirm the row for 'Bully Hill Restaurant' is in the same index row
venue_df

,name,categories,city,lat,lng,traffic,id
0,Boundary Breaks Vineyard,Winery,Lodi,42.628844,-76.864300,0,5310c5fc11d2285dadc69f6e
1,Herman J Wiemer Vineyard,Vineyard,Dundee,42.572324,-76.928225,0,4b896854f964a520513332e3
2,Goose Watch Winery,Winery,Romulus,42.758744,-76.771589,0,4bcf4e6841b9ef3bd72bf8e5
3,Cayuga Lake Creamery,Ice Cream Shop,Interlaken,42.616514,-76.695925,0,4bbcdf8aa0a0c9b6e9ea1a0f
4,Americana Vineyards & Winery,Vineyard,Interlaken,42.575900,-76.677117,0,4bef155bea570f4750e38ed2
5,Aurora Inn,Bed & Breakfast,Aurora,42.754605,-76.702821,0,4bc41c07dce4eee1f3f1719d
6,Lamoreaux Landing Wine Cellars,Vineyard,Lodi,42.576636,-76.858460,0,4c5dcaa785a1e21e12825a11
7,Seneca Farms,Ice Cream Shop,Penn Yan,42.655121,-77.071898,0,4bafdffff964a5200e283ce3
8,Scale House Brewery,Brewery,Hector,42.520547,-76.872937,0,5920940aee628b240b5e41e1
9,Taughannock Falls State Park,State / Provincial Park,Trumansburg,42.542489,-76.606600,0,4b57a728f964a520053a28e3


In [17]:
venue_df.at[74, 'city'] 

'Skaneateles'

In [18]:
# correct the values for 'Bully Hill Restaurant'
venue_df.at[74,'city'] = 'Hammondsport'
venue_df.at[74,'categories'] = 'Vineyard'

In [19]:
venue_df

,name,categories,city,lat,lng,traffic,id
0,Boundary Breaks Vineyard,Winery,Lodi,42.628844,-76.864300,0,5310c5fc11d2285dadc69f6e
1,Herman J Wiemer Vineyard,Vineyard,Dundee,42.572324,-76.928225,0,4b896854f964a520513332e3
2,Goose Watch Winery,Winery,Romulus,42.758744,-76.771589,0,4bcf4e6841b9ef3bd72bf8e5
3,Cayuga Lake Creamery,Ice Cream Shop,Interlaken,42.616514,-76.695925,0,4bbcdf8aa0a0c9b6e9ea1a0f
4,Americana Vineyards & Winery,Vineyard,Interlaken,42.575900,-76.677117,0,4bef155bea570f4750e38ed2
5,Aurora Inn,Bed & Breakfast,Aurora,42.754605,-76.702821,0,4bc41c07dce4eee1f3f1719d
6,Lamoreaux Landing Wine Cellars,Vineyard,Lodi,42.576636,-76.858460,0,4c5dcaa785a1e21e12825a11
7,Seneca Farms,Ice Cream Shop,Penn Yan,42.655121,-77.071898,0,4bafdffff964a5200e283ce3
8,Scale House Brewery,Brewery,Hector,42.520547,-76.872937,0,5920940aee628b240b5e41e1
9,Taughannock Falls State Park,State / Provincial Park,Trumansburg,42.542489,-76.606600,0,4b57a728f964a520053a28e3


In [20]:
fsq_df = venue_df.groupby('categories').count()
fsq_df

,name,city,lat,lng,traffic,id
categories,,,,,,
American Restaurant,4,4,4,4,4,4
Bagel Shop,1,1,1,1,1,1
Bakery,1,1,1,1,1,1
Bar,3,3,3,3,3,3
Bed & Breakfast,1,1,1,1,1,1
Breakfast Spot,2,2,2,2,2,2
Brewery,7,7,7,7,7,7
Burger Joint,2,2,2,2,2,2
Café,1,1,1,1,1,1


In [21]:
fsq_df.shape

(45, 6)

### Getting the details from the Foursquare API 

We're using a free sandbox account on Foursquare so we are limited to 50 premium calls per day. To deal with this, we'll build a csv file 50 rows at a time (per day over two days)

In [82]:
venue_df2 = venue_df.iloc[50:]
venue_df2

,name,categories,city,lat,lng,traffic,id
51,Ithaca Coffee Company,Coffee Shop,Ithaca,42.479640,-76.482095,0,4bb7369b941ad13a366020e3
52,Shortstop Deli,Sandwich Place,Ithaca,42.440531,-76.501531,0,4b5a3775f964a520e8b428e3
53,Lucifer Falls,Scenic Lookout,Ithaca,42.401306,-76.585556,0,4e3d4d4e7d8b0e96106735a6
54,Gimme! Coffee,Coffee Shop,Ithaca,42.444550,-76.499394,0,4b2fc858f964a520d1ef24e3
55,Collegetown Bagels,Bagel Shop,Ithaca,42.442537,-76.485179,0,4afd8fb4f964a520972822e3
56,Beebe Lake,Lake,Ithaca,42.451581,-76.477118,0,4c013ef1cf3aa59356b1ccb0
57,Lincoln Street Diner,Diner,Ithaca,42.452072,-76.496858,0,4b33b455f964a5205a1d25e3
58,Temple of Zeus,Sandwich Place,Ithaca,42.448884,-76.484106,0,4b4c9830f964a52012b726e3
59,Carriage House Cafe,American Restaurant,Ithaca,42.441996,-76.489878,0,4b5b323cf964a52059ea28e3
60,Skaneateles Lake,Lake,Skaneateles,42.940722,-76.425800,0,4aabc699f964a5200c5a20e3


### Getting a value for our traffic frequency from the Foursquare API

Foursquare no longer allow access to checkin counts so we'll use the sum of counts for photos and tips available for each venue. Getting these details require individual premium calls to the API - the account we are using to access the API is limited to 50 premium calls per day - so we'l break this up over two days and save the ID and traffic to a csv file.

### Day 1

In [25]:
# get the details for the first 50 venues and write them to a cvs file one row at a time

for venue_id in venue_df['id']:
    
        fsq_url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
        result = requests.get(fsq_url).json()
        traffic = result['response']['venue']['tips']['count'] + result['response']['venue']['photos']['count']
        wrData = [[venue_id , traffic]]
        with open('fl_venue_details.csv', 'a') as writeFile:
            writer = csv.writer(writeFile)
            writer.writerows(wrData)
            

### Day 2

In [23]:
# get the details of venues 50 through 100 into the csv file

for venue_id in venue_df2['id']:
    
        fsq_url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
        result = requests.get(fsq_url).json()
        traffic = result['response']['venue']['tips']['count'] + result['response']['venue']['photos']['count']
        wrData = [[venue_id , traffic]]
        with open('fl_venue_details.csv', 'a') as writeFile:
            writer = csv.writer(writeFile)
            writer.writerows(wrData)

### Now lets create a dataframe from the venue_details csv file

In [22]:
venue_det = pd.read_csv('fl_venue_details.csv')
venue_det.columns = ['id','traffic']
venue_det.head()

,id,traffic
0,4b896854f964a520513332e3,113
1,4bbcdf8aa0a0c9b6e9ea1a0f,85
2,4bef155bea570f4750e38ed2,159
3,4c5dcaa785a1e21e12825a11,82
4,4bc41c07dce4eee1f3f1719d,77


In [23]:
venue_det.describe()

,traffic
count,98.000000
mean,138.612245
std,159.232867
min,5.000000
25%,40.250000
50%,87.500000
75%,157.750000
max,1018.000000


Looking at the venue_det dataframe,above, we can see that we now have a wide range of rankings for traffic. Lets normalize the values for traffic to make them easier to visualize.

In [24]:
venue_det['traffic'] = venue_det['traffic']/venue_det['traffic'].max()

In [25]:
venue_det.describe()

,traffic
count,98.000000
mean,0.136161
std,0.156417
min,0.004912
25%,0.039538
50%,0.085953
75%,0.154961
max,1.000000


In [26]:
len(venue_df)

99

In [27]:
# merge the venue_df dataframe with the venue_det dataframe
venue_df = pd.merge(left=venue_df, right=venue_det, how='left', left_on='id', right_on='id')
venue_df = venue_df[['name', 'categories', 'city', 'lat', 'lng', 'traffic_y']] # we don't need the id column from here on
venue_df.rename(columns={'traffic_y':'traffic'},inplace=True)
venue_df

,name,categories,city,lat,lng,traffic
0,Boundary Breaks Vineyard,Winery,Lodi,42.628844,-76.864300,NaN
1,Herman J Wiemer Vineyard,Vineyard,Dundee,42.572324,-76.928225,0.111002
2,Goose Watch Winery,Winery,Romulus,42.758744,-76.771589,NaN
3,Cayuga Lake Creamery,Ice Cream Shop,Interlaken,42.616514,-76.695925,0.083497
4,Americana Vineyards & Winery,Vineyard,Interlaken,42.575900,-76.677117,0.156189
5,Aurora Inn,Bed & Breakfast,Aurora,42.754605,-76.702821,0.075639
6,Lamoreaux Landing Wine Cellars,Vineyard,Lodi,42.576636,-76.858460,0.080550
7,Seneca Farms,Ice Cream Shop,Penn Yan,42.655121,-77.071898,0.079568
8,Scale House Brewery,Brewery,Hector,42.520547,-76.872937,0.057957
9,Taughannock Falls State Park,State / Provincial Park,Trumansburg,42.542489,-76.606600,0.449902


Having a look at the merged dataframe above we see that we have multiple venues with NaN values for the traffic column. We'll replace those NaN values with the normalized minimum traffic value  from the traffic data frame (0.004912)

In [28]:
venue_df['traffic'] = np.where(venue_df['traffic'].isnull(), 0.004912, venue_df['traffic'])

In [29]:
venue_df.head()

,name,categories,city,lat,lng,traffic
0,Boundary Breaks Vineyard,Winery,Lodi,42.628844,-76.864300,0.004912
1,Herman J Wiemer Vineyard,Vineyard,Dundee,42.572324,-76.928225,0.111002
2,Goose Watch Winery,Winery,Romulus,42.758744,-76.771589,0.004912
3,Cayuga Lake Creamery,Ice Cream Shop,Interlaken,42.616514,-76.695925,0.083497
4,Americana Vineyards & Winery,Vineyard,Interlaken,42.575900,-76.677117,0.156189


### Let's make the values in venue_df uppercase to match the values in nys_df

In [30]:
venue_df = venue_df.apply(lambda x: x.astype(str).str.upper())
venue_df

,name,categories,city,lat,lng,traffic
0,BOUNDARY BREAKS VINEYARD,WINERY,LODI,42.628843745200676,-76.86429977416991,0.004912
1,HERMAN J WIEMER VINEYARD,VINEYARD,DUNDEE,42.57232383635794,-76.92822468545513,0.11100196463654224
2,GOOSE WATCH WINERY,WINERY,ROMULUS,42.75874440701328,-76.77158851465232,0.004912
3,CAYUGA LAKE CREAMERY,ICE CREAM SHOP,INTERLAKEN,42.616514168708825,-76.69592486877733,0.08349705304518663
4,AMERICANA VINEYARDS & WINERY,VINEYARD,INTERLAKEN,42.575900245716646,-76.67711687704586,0.15618860510805502
5,AURORA INN,BED & BREAKFAST,AURORA,42.754604930212125,-76.7028213419186,0.0756385068762279
6,LAMOREAUX LANDING WINE CELLARS,VINEYARD,LODI,42.57663558767305,-76.85845972795008,0.08055009823182711
7,SENECA FARMS,ICE CREAM SHOP,PENN YAN,42.655120501793505,-77.07189812008555,0.07956777996070727
8,SCALE HOUSE BREWERY,BREWERY,HECTOR,42.52054731619821,-76.87293749300326,0.05795677799607073
9,TAUGHANNOCK FALLS STATE PARK,STATE / PROVINCIAL PARK,TRUMANSBURG,42.542489,-76.6066,0.449901768172888


## Get data from New York State into a dataframe

In [31]:
nys_df = pd.read_csv('FL_Wineries__Breweries__and_Distilleries.csv')
nys_df.head()

,License Type Name,County Name (Licensee),Premises Name,Doing Business As (DBA),Actual Address of Premises (Address1),Additional Address Information (Address2),City,State,Zip,Latitude,Longitude
0,BREWERY,SCHUYLER,301 FRANKLIN STREET CAFE INC,NICKEL'S PIT BBQ,205 207 N FRANKLIN ST,NaN,WATKINS GLEN,NY,14891,0.000000,0.000000
1,WINERY,ONTARIO,5 KINDER FARM LLC,INSPIRE MOORE WINERY AND VINEYARD,197 N MAIN ST,NaN,NAPLES,NY,14512,42.625533,-77.395254
2,BREWERY,YATES,ABANDON BREWING COMPANY LLC,ABANDON BREWING COMPANY,2994 MERRITT HILL RD,NaN,PENN YAN,NY,14527,42.630805,-77.101060
3,WINERY,STEUBEN,AGNESS WINE CELLARS LLC,NaN,14157 W LAKE RD,NaN,PULTENEY,NY,14840,42.517200,-77.155940
4,WINERY,YATES,ALAN D HUNT,YATES CELLARS,3170 ROUTE 54A,NaN,BLUFF POINT,NY,14478,42.605786,-77.117031


## The dataset from NYS will need some wrangling

In [32]:
# Drop the address columns
nys_df = nys_df[['License Type Name','Premises Name','Doing Business As (DBA)', 'City', 'Latitude','Longitude']]
# Change the name of the columns to match the venue_df
nys_df = nys_df.rename(columns = {"License Type Name": "categories", 
                                  "Doing Business As (DBA)": "name",
                                  "City": "city",
                                  "Latitude": "lat",
                                  "Longitude": "lng"}) 

In [33]:
nys_df

,categories,Premises Name,name,city,lat,lng
0,BREWERY,301 FRANKLIN STREET CAFE INC,NICKEL'S PIT BBQ,WATKINS GLEN,0.000000,0.000000
1,WINERY,5 KINDER FARM LLC,INSPIRE MOORE WINERY AND VINEYARD,NAPLES,42.625533,-77.395254
2,BREWERY,ABANDON BREWING COMPANY LLC,ABANDON BREWING COMPANY,PENN YAN,42.630805,-77.101060
3,WINERY,AGNESS WINE CELLARS LLC,NaN,PULTENEY,42.517200,-77.155940
4,WINERY,ALAN D HUNT,YATES CELLARS,BLUFF POINT,42.605786,-77.117031
5,WINERY,AMERICANA VINEYARDS & WINERY INC,NaN,INTERLAKEN,42.575468,-76.677703
6,WINERY,ANTHONY ROAD WINE CO INC,NaN,PENN YANN,42.705734,-76.975292
7,DISTILLER,ANTLER RUN DISTILLING LLC,NaN,KEUKA PARK,0.000000,0.000000
8,WINERY,ARBOR HILL ASSOCIATES INC,NAPLES VALLEY WINE CELLARS,NAPLES,42.707382,-77.380418
9,WINERY,ATWATER ESTATE VINEYARDS LLC,NaN,BURDETT,42.479159,-76.869381


Looking at the data frame above we can see that we have some duplicates, so let's drop them

In [34]:
nys_df.drop_duplicates(['name', 'categories'], keep='first', inplace=True)

In [35]:
nys_df

,categories,Premises Name,name,city,lat,lng
0,BREWERY,301 FRANKLIN STREET CAFE INC,NICKEL'S PIT BBQ,WATKINS GLEN,0.000000,0.000000
1,WINERY,5 KINDER FARM LLC,INSPIRE MOORE WINERY AND VINEYARD,NAPLES,42.625533,-77.395254
2,BREWERY,ABANDON BREWING COMPANY LLC,ABANDON BREWING COMPANY,PENN YAN,42.630805,-77.101060
3,WINERY,AGNESS WINE CELLARS LLC,NaN,PULTENEY,42.517200,-77.155940
4,WINERY,ALAN D HUNT,YATES CELLARS,BLUFF POINT,42.605786,-77.117031
7,DISTILLER,ANTLER RUN DISTILLING LLC,NaN,KEUKA PARK,0.000000,0.000000
8,WINERY,ARBOR HILL ASSOCIATES INC,NAPLES VALLEY WINE CELLARS,NAPLES,42.707382,-77.380418
10,WINERY,AZURE HILL WINERY INC,AZURE HILL WINERY,HAMMONDSPORT,42.498098,-77.182490
11,BREWERY,BANDWAGON BREWERY LLC,BANDWAGON BREWERY,INTERLAKEN,42.616380,-76.728750
12,WINERY,BARTONS HARD CIDER LLC,BELLWETHER HARD CIDER,TRUMANSBURG,42.585809,-76.672456


In [36]:
len(nys_df)

149

In [37]:
nys_df.groupby('categories').count()

,Premises Name,name,city,lat,lng
categories,,,,,
BREWER,1,0,1,1,1
BREWERY,37,36,37,37,37
DISTILLER,12,11,12,12,12
DISTILLER,1,0,1,1,1
"DISTILLER ""C""",1,1,1,1,1
WINERY,96,95,96,96,96
WINERY / WINERY RETAIL,1,1,1,1,1


###  We'll use the DBA name as the venue name, so we'll need to use a simplified version of the Premises Name value for missing DBA values

In [38]:
nys_df['name'] = np.where(nys_df['name'].isnull(), nys_df['Premises Name'], nys_df['name'])

### We can't use data without lat/long so let's use the Lat/Long info for the city in which venues are located in place of missing values we'll get these from the Mapquest API

In [39]:
# we will be running this again later in the process so we'll delete and recreate it 
if os.path.exists("town_latlng.csv"):
  os.remove("town_latlng.csv")

# we'll create a csv file from the results for we don't have to keep going back to the API
for town in np.unique(nys_df['city']):
    
        mq_url = 'http://www.mapquestapi.com/geocoding/v1/address?key={}&city={}&state=NY'.format(mq_key, town)
        result = requests.get(mq_url).json()
        lat = result['results'][0]['locations'][0]['latLng']['lat']
        lng = result['results'][0]['locations'][0]['latLng']['lng']
        wrData = [[town, lat, lng]]
        with open('town_latlng.csv', 'a') as writeFile:
            writer = csv.writer(writeFile)
            writer.writerows(wrData)

In [40]:
# read the town Lat/Long data into a pandas dataframe that we will use here and 
# also eventually with our clustering data
town_latlng = pd.read_csv('town_latlng.csv')
town_latlng.columns = ['city', 'town_lat', 'town_lng']
town_latlng.head()

,city,town_lat,town_lng
0,AUBURN,42.931778,-76.566140
1,AURORA,42.753982,-76.702410
2,BLOOMFIELD,42.899855,-77.426148
3,BLUFF POINT,42.614201,-77.103897
4,BRANCHPORT,42.598598,-77.154198


### First we'll temporarily merge the town_latlng and nys_df dataframes

In [41]:
temp_ny_df = pd.merge(left=nys_df,right=town_latlng, left_on='city', right_on='city').copy()

In [42]:
temp_ny_df

,categories,Premises Name,name,city,lat,lng,town_lat,town_lng
0,BREWERY,301 FRANKLIN STREET CAFE INC,NICKEL'S PIT BBQ,WATKINS GLEN,0.000000,0.000000,42.380545,-76.873456
1,WINERY,CASTEL GRISCH MANAGEMENT LLC,CASTEL GRISCH,WATKINS GLEN,42.397283,-76.905411,42.380545,-76.873456
2,WINERY,LAKEWOOD VINEYARDS INC,LAKEWOOD VINEYARDS,WATKINS GLEN,42.865428,-77.220888,42.380545,-76.873456
3,BREWERY,SENECA LODGE INC,SENECA LODGE CRAFT BREWING,WATKINS GLEN,0.000000,0.000000,42.380545,-76.873456
4,WINERY,5 KINDER FARM LLC,INSPIRE MOORE WINERY AND VINEYARD,NAPLES,42.625533,-77.395254,42.613423,-77.403828
5,WINERY,ARBOR HILL ASSOCIATES INC,NAPLES VALLEY WINE CELLARS,NAPLES,42.707382,-77.380418,42.613423,-77.403828
6,BREWERY,ABANDON BREWING COMPANY LLC,ABANDON BREWING COMPANY,PENN YAN,42.630805,-77.101060,42.660945,-77.053647
7,WINERY,BLACK WALNUT VINEYARDS LLC,SERENITY VINEYARDS,PENN YAN,42.715735,-76.971718,42.660945,-77.053647
8,WINERY,CK CELLARS LLC,TORREY RIDGE WINERY EARLE ESTATES MEADERY,PENN YAN,42.628820,-76.948710,42.660945,-77.053647
9,WINERY,FOX RUN VINEYARDS INC,FOX RUN,PENN YAN,42.732376,-76.971804,42.660945,-77.053647


In [43]:
temp_ny_df['lat'] = np.where(temp_ny_df['lat']==0, temp_ny_df['town_lat'], temp_ny_df['lat'])
temp_ny_df['lat'] = np.where(temp_ny_df['lat'].isnull(), temp_ny_df['town_lat'], temp_ny_df['lat'])
temp_ny_df['lng'] = np.where(temp_ny_df['lng']==0, temp_ny_df['town_lng'], temp_ny_df['lng'])
temp_ny_df['lng'] = np.where(temp_ny_df['lng'].isnull(), temp_ny_df['town_lng'], temp_ny_df['lng'])

In [44]:
nys_df = temp_ny_df[['name', 'categories', 'city', 'lat', 'lng']]

In [45]:
nys_df

,name,categories,city,lat,lng
0,NICKEL'S PIT BBQ,BREWERY,WATKINS GLEN,42.380545,-76.873456
1,CASTEL GRISCH,WINERY,WATKINS GLEN,42.397283,-76.905411
2,LAKEWOOD VINEYARDS,WINERY,WATKINS GLEN,42.865428,-77.220888
3,SENECA LODGE CRAFT BREWING,BREWERY,WATKINS GLEN,42.380545,-76.873456
4,INSPIRE MOORE WINERY AND VINEYARD,WINERY,NAPLES,42.625533,-77.395254
5,NAPLES VALLEY WINE CELLARS,WINERY,NAPLES,42.707382,-77.380418
6,ABANDON BREWING COMPANY,BREWERY,PENN YAN,42.630805,-77.101060
7,SERENITY VINEYARDS,WINERY,PENN YAN,42.715735,-76.971718
8,TORREY RIDGE WINERY EARLE ESTATES MEADERY,WINERY,PENN YAN,42.628820,-76.948710
9,FOX RUN,WINERY,PENN YAN,42.732376,-76.971804


In [46]:
len(nys_df)

148

#### The NY State data does not contain values that we can use for a traffic indicator so we'll use the lowest value of the nomralized traffic data from the Foursquare API

In [47]:
nys_df.insert(5, "traffic", 0.004912)
nys_df.head()

,name,categories,city,lat,lng,traffic
0,NICKEL'S PIT BBQ,BREWERY,WATKINS GLEN,42.380545,-76.873456,0.004912
1,CASTEL GRISCH,WINERY,WATKINS GLEN,42.397283,-76.905411,0.004912
2,LAKEWOOD VINEYARDS,WINERY,WATKINS GLEN,42.865428,-77.220888,0.004912
3,SENECA LODGE CRAFT BREWING,BREWERY,WATKINS GLEN,42.380545,-76.873456,0.004912
4,INSPIRE MOORE WINERY AND VINEYARD,WINERY,NAPLES,42.625533,-77.395254,0.004912


In [48]:
nys_cnt = nys_df.groupby('categories').count()
nys_cnt

,name,city,lat,lng,traffic
categories,,,,,
BREWER,1,1,1,1,1
BREWERY,37,37,37,37,37
DISTILLER,12,12,12,12,12
DISTILLER,1,1,1,1,1
"DISTILLER ""C""",1,1,1,1,1
WINERY,95,95,95,95,95
WINERY / WINERY RETAIL,1,1,1,1,1


In [49]:
venue_df.head()

,name,categories,city,lat,lng,traffic
0,BOUNDARY BREAKS VINEYARD,WINERY,LODI,42.628843745200676,-76.86429977416991,0.004912
1,HERMAN J WIEMER VINEYARD,VINEYARD,DUNDEE,42.57232383635794,-76.92822468545513,0.11100196463654224
2,GOOSE WATCH WINERY,WINERY,ROMULUS,42.75874440701328,-76.77158851465232,0.004912
3,CAYUGA LAKE CREAMERY,ICE CREAM SHOP,INTERLAKEN,42.616514168708825,-76.69592486877733,0.08349705304518663
4,AMERICANA VINEYARDS & WINERY,VINEYARD,INTERLAKEN,42.575900245716646,-76.67711687704586,0.15618860510805502


### Merge the dataframes that we've built from the Foursquare API data and the NY State data; then get the data into a usable state by replacing categorical value and normalizing.

In [50]:
venue_df = pd.concat([venue_df, nys_df], axis=0)
venue_df = venue_df.reset_index(drop=True)
venue_df

,name,categories,city,lat,lng,traffic
0,BOUNDARY BREAKS VINEYARD,WINERY,LODI,42.628843745200676,-76.86429977416991,0.004912
1,HERMAN J WIEMER VINEYARD,VINEYARD,DUNDEE,42.57232383635794,-76.92822468545513,0.11100196463654224
2,GOOSE WATCH WINERY,WINERY,ROMULUS,42.75874440701328,-76.77158851465232,0.004912
3,CAYUGA LAKE CREAMERY,ICE CREAM SHOP,INTERLAKEN,42.616514168708825,-76.69592486877733,0.08349705304518663
4,AMERICANA VINEYARDS & WINERY,VINEYARD,INTERLAKEN,42.575900245716646,-76.67711687704586,0.15618860510805502
5,AURORA INN,BED & BREAKFAST,AURORA,42.754604930212125,-76.7028213419186,0.0756385068762279
6,LAMOREAUX LANDING WINE CELLARS,VINEYARD,LODI,42.57663558767305,-76.85845972795008,0.08055009823182711
7,SENECA FARMS,ICE CREAM SHOP,PENN YAN,42.655120501793505,-77.07189812008555,0.07956777996070727
8,SCALE HOUSE BREWERY,BREWERY,HECTOR,42.52054731619821,-76.87293749300326,0.05795677799607073
9,TAUGHANNOCK FALLS STATE PARK,STATE / PROVINCIAL PARK,TRUMANSBURG,42.542489,-76.6066,0.449901768172888


In [51]:
len(venue_df)

247

In [52]:
venue_df.sort_values(by=['name'])

,name,categories,city,lat,lng,traffic
230,"(DISTILLER ""C"")",DISTILLER,ROMULUS,42.2199,-76.9742,0.004912
79,A&W ALL-AMERICAN FOOD,BURGER JOINT,CORTLAND,42.57609889473593,-76.21703262545307,0.07170923379174853
105,ABANDON BREWING COMPANY,BREWERY,PENN YAN,42.6308,-77.1011,0.004912
66,ABBOTT'S FROZEN CUSTARD,ICE CREAM SHOP,CANANDAIGUA,42.87871780998024,-77.26408325158883,0.011787819253438114
118,AGNESS WINE CELLARS LLC,WINERY,PULTENEY,42.5172,-77.1559,0.004912
4,AMERICANA VINEYARDS & WINERY,VINEYARD,INTERLAKEN,42.575900245716646,-76.67711687704586,0.15618860510805502
96,ANIELLO'S PIZZERIA,PIZZA PLACE,CORNING,42.142932482364614,-77.05171191610434,0.11198428290766209
120,ANTLER RUN DISTILLING LLC,DISTILLER,KEUKA PARK,42.6151,-77.0919,0.004912
97,ATLAS BRICK OVEN PIZZERIA,PIZZA PLACE,CORNING,42.14326782991742,-77.05302946139088,0.10707269155206287
16,ATWATER ESTATE VINEYARDS,VINEYARD,BURDETT,42.478273911139524,-76.86978286068678,0.11394891944990176


### Replace categorical data

In order get some meaningful insight from our traffic data, we'll need to convert the categorical data into numeric data. We only really care about three groups of categories for this analysis - 1) alcoholic beverage producers like wineries, breweris, and distilleries, 2) scenic/tourist attractions, and 3) other businesses with data available from Foursquare's API (i.e. everything else). 

In [53]:
# add columns to numerically identify venue category groups
venue_df.insert(3, "distillery", 0)
venue_df.insert(4,"winery", 0)
venue_df.insert(5,"brewery", 0)
venue_df.insert(6, "scenic", 0)
venue_df.insert(7, "other", 0)

In [54]:
# select slices of the dataframes based on categories
distillery_df = venue_df.loc[venue_df['categories'].isin(['DISTILLER','DISTILLER "C"'])]
winery_df = venue_df.loc[venue_df['categories'].isin(['WINERY', 'VINEYARD','WINERY / WINERY RETAIL'])]
brewery_df = venue_df.loc[venue_df['categories'].isin(['BREWERY','BREWER'])]
scenic_df = venue_df.loc[venue_df['categories'].isin(['FARM','TRAIL','PARK', 'SCENIC LOOKOUT', 'WATERFALL'])]

In [55]:
# since we only care about distilleries, wineries, breweries, and scenic attractions, throw all of the others together
interesting_df = pd.concat([distillery_df, winery_df, brewery_df, scenic_df], axis=0)
interesting_cats = interesting_df['categories'].tolist()
other_df = venue_df.query('categories not in @interesting_cats').copy()

In [56]:
# set each slice to 1
distillery_df.loc[:,'distillery'] = 1
winery_df.loc[:,'winery'] = 1
brewery_df.loc[:,'brewery'] = 1
scenic_df.loc[:,'scenic'] = 1
other_df.loc[:,'other'] = 1

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [57]:
print("There are {} wineries, {} breweries, {} distilleries, {} scenic venues, and \n {} other business within our combined dataframe".format(
        len(winery_df), len(brewery_df), len(distillery_df), len(scenic_df), len(other_df)
            )
     )

There are 106 wineries, 45 breweries, 13 distilleries, 10 scenic venues, and 
 73 other business within our combined dataframe


In [58]:
venue_df = pd.concat([distillery_df, winery_df, brewery_df, scenic_df, other_df], axis=0)


In [59]:
venue_df[['distillery', 'winery', 'brewery', 'scenic', 'other', 'traffic']] = venue_df[['distillery', 'winery', 'brewery', 'scenic', 'other', 'traffic']].apply(pd.to_numeric)

venue_df.dtypes

name           object
categories     object
city           object
distillery      int64
winery          int64
brewery         int64
scenic          int64
other           int64
lat            object
lng            object
traffic       float64
dtype: object

In [60]:
venue_df

,name,categories,city,distillery,winery,brewery,scenic,other,lat,lng,traffic
120,ANTLER RUN DISTILLING LLC,DISTILLER,KEUKA PARK,1,0,0,0,0,42.6151,-77.0919,0.004912
123,KROOKED TUSKER DISTILLERY,DISTILLER,HAMMONDSPORT,1,0,0,0,0,42.491,-77.1874,0.004912
147,SPYGLASS SPIRITS,DISTILLER,GENEVA,1,0,0,0,0,42.8796,-76.984,0.004912
153,DAMIANI WINE CELLARS,DISTILLER,HECTOR,1,0,0,0,0,42.4954,-76.8805,0.004912
185,TUITION DISTILLING COMPANY,DISTILLER,CANANDAIGUA,1,0,0,0,0,42.9016,-77.3054,0.004912
191,DILL'S RUN DISTILLERY,DISTILLER,UNION SPRINGS,1,0,0,0,0,42.8044,-76.7037,0.004912
203,BARRINGTON DISTILLERS,DISTILLER,DUNDEE,1,0,0,0,0,42.5205,-77.046,0.004912
215,MYER FARM DISTILLERS,DISTILLER,OVID,1,0,0,0,0,42.6691,-76.7307,0.004912
221,SIX MILE CREEK VINEYARD,DISTILLER,ITHACA,1,0,0,0,0,42.4158,-76.4553,0.004912
226,KNAPP VINEYARDS,"DISTILLER ""C""",ROMULUS,1,0,0,0,0,42.7658,-76.7832,0.004912


In [61]:
print('The venue dataframe has {} towns and {} venues.'.format(
        len(venue_df['city'].unique()),
        venue_df.shape[0]
    )
)

The venue dataframe has 51 towns and 247 venues.


Given the sparsity of the data available from the Foursquare API for the Finger Lakes region, we will only use this data to an overview of traffic data based on user activity. Licensing data from the New York State Liquor Authority will be used to show density of existing distilleries, wineries, and breweries. 

In [62]:
traffic_df = venue_df[['city','distillery', 'winery', 'brewery','scenic','other','traffic']]
traffic_df = traffic_df.groupby(['city']).sum()


In [63]:
traffic_df.head()

,distillery,winery,brewery,scenic,other,traffic
city,,,,,,
AUBURN,0,1,3,0,3,0.452850
AURORA,0,1,0,0,1,0.080551
BLOOMFIELD,0,2,1,0,0,0.014736
BLUFF POINT,0,1,0,0,0,0.004912
BRANCHPORT,0,1,0,0,0,0.004912


### Now we need to normalize the venue counts for each category 

In [64]:
# first keep the original count values for future use in analyzing the makup of clustered data
venue_counts = traffic_df.copy()

In [65]:
# get a normalize frequency value for each category
traffic_df['distillery'] = traffic_df['distillery']/traffic_df['distillery'].max()
traffic_df['winery'] = traffic_df['winery']/traffic_df['winery'].max()
traffic_df['brewery'] = traffic_df['brewery']/traffic_df['brewery'].max()
traffic_df['scenic'] = traffic_df['scenic']/traffic_df['scenic'].max()
traffic_df['other'] = traffic_df['other']/traffic_df['other'].max()

In [66]:
traffic_df.reset_index(level=0, inplace=True)
venue_counts.reset_index(level=0, inplace=True)

In [67]:
venue_counts.head()

,city,distillery,winery,brewery,scenic,other,traffic
0,AUBURN,0,1,3,0,3,0.452850
1,AURORA,0,1,0,0,1,0.080551
2,BLOOMFIELD,0,2,1,0,0,0.014736
3,BLUFF POINT,0,1,0,0,0,0.004912
4,BRANCHPORT,0,1,0,0,0,0.004912


In [68]:
traffic_df.head()

,city,distillery,winery,brewery,scenic,other,traffic
0,AUBURN,0.0,0.090909,0.6,0.0,0.176471,0.452850
1,AURORA,0.0,0.090909,0.0,0.0,0.058824,0.080551
2,BLOOMFIELD,0.0,0.181818,0.2,0.0,0.000000,0.014736
3,BLUFF POINT,0.0,0.090909,0.0,0.0,0.000000,0.004912
4,BRANCHPORT,0.0,0.090909,0.0,0.0,0.000000,0.004912


In [69]:
cluster_traffic_df = traffic_df[['distillery', 'winery', 'brewery','scenic','other', 'traffic']]

In [70]:
#k_clusters = 5
k_means = KMeans(init="k-means++", n_clusters=5, n_init=12)
k_means.fit(cluster_traffic_df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=12, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [71]:
k_means.labels_

array([3, 0, 0, 0, 0, 0, 4, 3, 0, 3, 0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0,
       1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 0, 0, 0, 4, 4, 0, 0, 0,
       3, 4, 0, 0, 0, 3, 0], dtype=int32)

In [72]:
# add the k_means labels to the traffic_df and venue_counts dataframes
traffic_df.insert(0, 'cluster labels', k_means.labels_)
venue_counts.insert(0, 'cluster labels', k_means.labels_)

### we'll hit the Mapquest API again to be sure that we have all the coodinates we need

In [73]:
# we will be running this again later in the process so we'll delete and recreate it 
if os.path.exists("town_latlng.csv"):
  os.remove("town_latlng.csv")

# we'll create a csv file from the results for we don't have to keep going back to the API
for town in np.unique(traffic_df['city']):
    
        mq_url = 'http://www.mapquestapi.com/geocoding/v1/address?key={}&city={}&state=NY'.format(mq_key, town)
        result = requests.get(mq_url).json()
        lat = result['results'][0]['locations'][0]['latLng']['lat']
        lng = result['results'][0]['locations'][0]['latLng']['lng']
        wrData = [[town, lat, lng]]
        with open('town_latlng.csv', 'a') as writeFile:
            writer = csv.writer(writeFile)
            writer.writerows(wrData)

In [74]:
# read the town Lat/Long data into a pandas dataframe
town_latlng = pd.read_csv('town_latlng.csv')
town_latlng.columns = ['city', 'lat', 'lng']
town_latlng.head()

,city,lat,lng
0,AURORA,42.753982,-76.702410
1,BLOOMFIELD,42.899855,-77.426148
2,BLUFF POINT,42.614201,-77.103897
3,BRANCHPORT,42.598598,-77.154198
4,BURDETT,42.417132,-76.849486


### Merge the Mapquest API sourced data in town_latlng with the traffic_df dataframe so we can map the clusters

In [75]:
traffic_df = pd.merge(left=traffic_df,right=town_latlng, left_on='city', right_on='city')

traffic_df.head()

,cluster labels,city,distillery,winery,brewery,scenic,other,traffic,lat,lng
0,0,AURORA,0.0,0.090909,0.0,0.0,0.058824,0.080551,42.753982,-76.702410
1,0,BLOOMFIELD,0.0,0.181818,0.2,0.0,0.000000,0.014736,42.899855,-77.426148
2,0,BLUFF POINT,0.0,0.090909,0.0,0.0,0.000000,0.004912,42.614201,-77.103897
3,0,BRANCHPORT,0.0,0.090909,0.0,0.0,0.000000,0.004912,42.598598,-77.154198
4,0,BURDETT,0.0,0.363636,0.2,0.0,0.000000,0.133597,42.417132,-76.849486


In [76]:
# we can drop the city column as we are going to use this to analyze cluster make-up
cluster_counts = venue_counts[['cluster labels','distillery', 'winery', 'brewery', 'scenic', 'other', 'traffic']]
cluster_counts.groupby(['cluster labels'], axis=0).sum()

,distillery,winery,brewery,scenic,other,traffic
cluster labels,,,,,,
0,0,36,15,2,31,2.712201
1,1,2,4,6,17,4.789786
2,4,45,12,0,6,0.999041
3,1,10,13,2,17,3.220049
4,7,13,1,0,2,0.166020


In [77]:
# have a look at which towns belong to which clusters - along with venue counts and traffic
town_counts = venue_counts.groupby(['city'], axis=0).sum()
town_counts.sort_values(by=['cluster labels'])

,cluster labels,distillery,winery,brewery,scenic,other,traffic
city,,,,,,,
LAFAYETTE,0,0,0,0,1,0,0.155206
LANSING,0,0,0,0,0,1,0.067780
LOCKE,0,0,1,0,0,0,0.004912
LODI,0,0,5,1,0,0,0.105110
MANCHESTER,0,0,0,1,0,0,0.004912
MILO,0,0,1,0,0,0,0.004912
MONTOUR FALLS,0,0,0,0,1,0,0.089391
NAN,0,0,0,0,0,2,0.009824
NAPLES,0,0,2,1,0,2,0.089392


In [137]:
# Output to csv for use in our report
cluster_counts.groupby(['cluster labels'], axis=0).sum().to_csv('cluster_counts.csv', encoding = 'utf-8', index='False')
town_counts.sort_values(by=['cluster labels', 'city']).to_csv('town_clusters.csv', encoding = 'utf-8', index='False')

## Time to build a base map and have a look at the distribution of our venues from the Foursquare API and the New York State data 

In [78]:

map_FL = folium.Map(location=[latitude, longitude], zoom_start=9)

# add Foursquare venue_df markers to map
for lat, lng, name, categories in zip(venue_df['lat'], venue_df['lng'], venue_df['name'], venue_df['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_FL)  
# add New York State nys_df markers to map

for lat, lng, name, categories in zip(nys_df['lat'], nys_df['lng'], nys_df['name'], nys_df['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_FL)  

       
    
map_FL

## Now let's map out our venue and traffic cluster data


In [79]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9)


# set color scheme for the clusters
cluster_colors = ['red', 'blue', 'orange', 'green', 'purple']

# add markers to the map
markers_colors = []
for lat, lon, town, cluster in zip(traffic_df['lat'], traffic_df['lng'], traffic_df['city'], traffic_df['cluster labels']):
    label = folium.Popup(str(town) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=cluster_colors[cluster],
        fill=True,
        fill_color=cluster_colors[cluster],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters